In [1]:
import pandas as pd
import zipfile
import os
import pandas as pd

# Load the raw and core data
raw_recipes = pd.read_csv ('raw-data_recipe.csv')
raw_interactions = pd.read_csv('raw-data_interaction.csv')
core_recipes = pd.read_csv('core-data_recipe.csv')
core_interactions_train = pd.read_csv('core-data-train_rating.csv')
core_interactions_valid = pd.read_csv('core-data-valid_rating.csv')
core_interactions_test = pd.read_csv('core-data-test_rating.csv')


# Display the first few rows of the datasets
print(core_recipes.head())
print(core_interactions_train.head())
print(core_interactions_valid.head())
print(core_interactions_test.head())



   recipe_id                              recipe_name  \
0     240488        Pork Loin, Apples, and Sauerkraut   
1     218939         Foolproof Rosemary Chicken Wings   
2      87211                    Chicken Pesto Paninis   
3     245714                       Potato Bacon Pizza   
4     218545  Latin-Inspired Spicy Cream Chicken Stew   

                                           image_url  \
0  https://images.media-allrecipes.com/userphotos...   
1  https://images.media-allrecipes.com/userphotos...   
2  https://images.media-allrecipes.com/userphotos...   
3  https://images.media-allrecipes.com/userphotos...   
4  https://images.media-allrecipes.com/userphotos...   

                                         ingredients  \
0  sauerkraut drained^Granny Smith apples sliced^...   
1  chicken wings^sprigs rosemary^head garlic^oliv...   
2  focaccia bread quartered^prepared basil pesto^...   
3  red potatoes^strips bacon^Sauce:^heavy whippin...   
4  skinless boneless chicken breast halv

In [2]:
print(core_recipes.columns)

# Data Preprocessing
# Handle missing values if any
core_recipes.fillna('', inplace=True)

# If 'category' column exists, encode it
if 'category' in core_recipes.columns:
    label_encoder = LabelEncoder()
    core_recipes['category'] = label_encoder.fit_transform(core_recipes['category'])
else:
    print("'category' column does not exist in core_recipes")

# Normalize numerical features if they exist
numerical_features = ['rating', 'review_nums', 'nutrition']
existing_numerical_features = [feature for feature in numerical_features if feature in core_recipes.columns]

if existing_numerical_features:
    scaler = StandardScaler()
    core_recipes[existing_numerical_features] = scaler.fit_transform(core_recipes[existing_numerical_features])
else:
    print("No numerical features found in core_recipes")

Index(['recipe_id', 'recipe_name', 'image_url', 'ingredients',
       'cooking_directions', 'nutritions'],
      dtype='object')
'category' column does not exist in core_recipes
No numerical features found in core_recipes


In [3]:
# Feature Engineering
# Example of creating a new feature for user preferences
user_preferences = core_interactions_train.groupby('user_id')['recipe_id'].apply(list).reset_index()
user_preferences.columns = ['user_id', 'preferred_recipes']


In [4]:
!pip install implicit

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

# Load the raw and core data
raw_recipes = pd.read_csv('raw-data_recipe.csv')
raw_interactions = pd.read_csv('raw-data_interaction.csv')
core_recipes = pd.read_csv('core-data_recipe.csv')
core_interactions_train = pd.read_csv('core-data-train_rating.csv')
core_interactions_valid = pd.read_csv('core-data-valid_rating.csv')
core_interactions_test = pd.read_csv('core-data-test_rating.csv')

# Create mappings for user and recipe IDs to ensure they are within matrix dimensions
user_id_map = {user_id: idx for idx, user_id in enumerate(core_interactions_train['user_id'].unique())}
recipe_id_map = {recipe_id: idx for idx, recipe_id in enumerate(core_interactions_train['recipe_id'].unique())}

# Map the user and recipe IDs
core_interactions_train['user_idx'] = core_interactions_train['user_id'].map(user_id_map)
core_interactions_train['recipe_idx'] = core_interactions_train['recipe_id'].map(recipe_id_map)

# Get the number of unique users and recipes
n_users = len(user_id_map)
n_recipes = len(recipe_id_map)

# Create a sparse matrix
train_matrix = csr_matrix((core_interactions_train['rating'], 
                           (core_interactions_train['user_idx'], core_interactions_train['recipe_idx'])), 
                          shape=(n_users, n_recipes))

# Verify the shape of the matrix
print(f"Train matrix shape: {train_matrix.shape}")

# Display a portion of the sparse matrix for verification
print(train_matrix[:5, :5].todense())


Train matrix shape: (19362, 23041)
[[5 4 5 0 0]
 [0 0 0 4 0]
 [0 0 0 0 5]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [9]:
# SVD and Recommendations:

import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

# Load the raw and core data
raw_recipes = pd.read_csv('raw-data_recipe.csv')
raw_interactions = pd.read_csv('raw-data_interaction.csv')
core_recipes = pd.read_csv('core-data_recipe.csv')
core_interactions_train = pd.read_csv('core-data-train_rating.csv')
core_interactions_valid = pd.read_csv('core-data-valid_rating.csv')
core_interactions_test = pd.read_csv('core-data-test_rating.csv')

# Create mappings for user and recipe IDs to ensure they are within matrix dimensions
user_id_map = {user_id: idx for idx, user_id in enumerate(core_interactions_train['user_id'].unique())}
recipe_id_map = {recipe_id: idx for idx, recipe_id in enumerate(core_interactions_train['recipe_id'].unique())}

# Map the user and recipe IDs
core_interactions_train['user_idx'] = core_interactions_train['user_id'].map(user_id_map)
core_interactions_train['recipe_idx'] = core_interactions_train['recipe_id'].map(recipe_id_map)

# Get the number of unique users and recipes
n_users = len(user_id_map)
n_recipes = len(recipe_id_map)

# Create a sparse matrix
train_matrix = csr_matrix((core_interactions_train['rating'], 
                           (core_interactions_train['user_idx'], core_interactions_train['recipe_idx'])), 
                          shape=(n_users, n_recipes))

# Apply TruncatedSVD
svd = TruncatedSVD(n_components=50, random_state=42)
train_matrix_reduced = svd.fit_transform(train_matrix)

# Compute the reconstructed matrix
train_matrix_reconstructed = svd.inverse_transform(train_matrix_reduced)

# Function to recommend top N recipes for a given user
def recommend_recipes(user_idx, train_matrix_reconstructed, n_recommendations=5):
    # Get the user's ratings from the reconstructed matrix
    user_ratings = train_matrix_reconstructed[user_idx]
    
    # Get the indices of the top N recommended recipes
    recommended_indices = np.argsort(user_ratings)[::-1][:n_recommendations]
    
    return recommended_indices

# Example: Recommend top 5 recipes for the first user
user_idx = 0
recommendations = recommend_recipes(user_idx, train_matrix_reconstructed, n_recommendations=5)
print(f"Top 5 recommendations for user {user_idx}: {recommendations}")


Top 5 recommendations for user 0: [ 614 2000  972 1340  476]


In [10]:
# Evaluation :

# Map validation and test interactions to the matrix indices
core_interactions_valid['user_idx'] = core_interactions_valid['user_id'].map(user_id_map)
core_interactions_valid['recipe_idx'] = core_interactions_valid['recipe_id'].map(recipe_id_map)

core_interactions_test['user_idx'] = core_interactions_test['user_id'].map(user_id_map)
core_interactions_test['recipe_idx'] = core_interactions_test['recipe_id'].map(recipe_id_map)

# Ensure indices are valid integers
core_interactions_valid = core_interactions_valid.dropna(subset=['user_idx', 'recipe_idx'])
core_interactions_valid['user_idx'] = core_interactions_valid['user_idx'].astype(int)
core_interactions_valid['recipe_idx'] = core_interactions_valid['recipe_idx'].astype(int)

core_interactions_test = core_interactions_test.dropna(subset=['user_idx', 'recipe_idx'])
core_interactions_test['user_idx'] = core_interactions_test['user_idx'].astype(int)
core_interactions_test['recipe_idx'] = core_interactions_test['recipe_idx'].astype(int)

# Function to get the predicted rating for a specific user and recipe
def get_predicted_rating(user_idx, recipe_idx, train_matrix_reconstructed):
    return train_matrix_reconstructed[user_idx, recipe_idx]

# Evaluate on validation set
valid_predictions = []
valid_actuals = []

for _, row in core_interactions_valid.iterrows():
    user_idx = row['user_idx']
    recipe_idx = row['recipe_idx']
    
    predicted_rating = get_predicted_rating(user_idx, recipe_idx, train_matrix_reconstructed)
    valid_predictions.append(predicted_rating)
    valid_actuals.append(row['rating'])

# Calculate RMSE for validation set
valid_rmse = np.sqrt(mean_squared_error(valid_actuals, valid_predictions))
print(f"Validation RMSE: {valid_rmse}")

# Evaluate on test set
test_predictions = []
test_actuals = []

for _, row in core_interactions_test.iterrows():
    user_idx = row['user_idx']
    recipe_idx = row['recipe_idx']
    
    predicted_rating = get_predicted_rating(user_idx, recipe_idx, train_matrix_reconstructed)
    test_predictions.append(predicted_rating)
    test_actuals.append(row['rating'])

# Calculate RMSE for test set
test_rmse = np.sqrt(mean_squared_error(test_actuals, test_predictions))
print(f"Test RMSE: {test_rmse}")


Validation RMSE: 4.554619335959658
Test RMSE: 4.574534306956035
